In [2]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

In [3]:
import spacy

In [4]:
d=read_file("dataset/Administration.txt")

In [5]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [6]:
nlp.max_length = 1198623

In [7]:
def separate_punt(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%^&*()--._{|}:+-=<>?\t\n,;`/\\[]']

In [8]:
# d = read_file("moby_dick_four_chapters.txt")

In [9]:
tokens = separate_punt(d)

In [10]:
print(len(tokens)
     )

30959


In [13]:
train_len = 4 + 1
text_sequence = []

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequence.append(seq)
    

In [14]:
text_sequence[1]

['+852', '9060', '8866', 'email', 'liukamay@gmail.com']

In [15]:
" ".join(text_sequence[0])

'tel +852 9060 8866 email'

In [16]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequence)

In [18]:
sequence = tokenizer.texts_to_sequences(text_sequence)

In [19]:
# sequence[2]

In [20]:
# tokenizer.index_word

In [21]:
for i in sequence[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

1287 : tel
2364 : +852
4297 : 9060
4296 : 8866
316 : email


In [22]:
tokenizer.word_counts

OrderedDict([('tel', 16),
             ('+852', 7),
             ('9060', 3),
             ('8866', 4),
             ('email', 80),
             ('liukamay@gmail.com', 5),
             ('address', 40),
             ('flat', 30),
             ('1015', 5),
             ('to', 3315),
             ('obtain', 25),
             ('personal', 380),
             ('assistant', 485),
             ('secretary', 270),
             ('which', 110),
             ('is', 215),
             ('a', 765),
             ('strong', 100),
             ('dedication', 5),
             ('the', 3915),
             ('guest', 45),
             ('satisfaction', 15),
             ('and', 10510),
             ('an', 185),
             ('excellent', 75),
             ('communication', 115),
             ('skills', 170),
             ('professionalism', 20),
             ('can', 40),
             ('be', 55),
             ('fully', 25),
             ('utilized', 5),
             ('ecrent', 5),
             ('worldwide', 45

In [23]:
vocabulary_size = len(tokenizer.word_counts)

In [24]:
vocabulary_size

4297

In [25]:
type(sequence)

list

In [26]:
import numpy as np
sequences = np.array(sequence)

In [27]:
sequences

array([[1287, 2364, 4297, 4296,  316],
       [2364, 4297, 4296,  316, 2366],
       [4297, 4296,  316, 2366,  624],
       ...,
       [  19, 1099,  604, 1286,    6],
       [1099,  604, 1286,    6, 2365],
       [ 604, 1286,    6, 2365, 1644]])

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
from keras.utils import to_categorical

In [30]:
X=sequences[:,:-1]

In [31]:
y=sequences[:, -1]

In [32]:
y=to_categorical(y, num_classes=vocabulary_size+1)

In [33]:
seq_len=X.shape[1]

In [34]:
X.shape

(30954, 4)

In [35]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [46]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    
    model.add(Dense(vocabulary_size, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [51]:
print(vocabulary_size, seq_len)
model=create_model(vocabulary_size+1, seq_len)

4297 4
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 4, 4)              17192     
_________________________________________________________________
lstm_7 (LSTM)                (None, 4, 50)             11000     
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 4298)              219198    
Total params: 270,140
Trainable params: 270,140
Non-trainable params: 0
_________________________________________________________________


In [52]:
from pickle import dump, load
model.save("mymobe_dick.h5")
dump(tokenizer, open('my_simpletokenizer', 'wb'))

In [53]:
from keras.preprocessing.sequence import pad_sequences
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text 
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' '+pred_word
        
        output_text.append(pred_word)
    
    return " ".join(output_text)

In [56]:
import random
random.seed(32)
random_pick = random.randint(0, len(text_sequence))
random_seed_text = text_sequence[random_pick]
seed_text = ' '.join(random_seed_text)
# print(seed_text)
seed_text = "Heavily involved in the integration of in-house operation system"
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=2)

'offer rays'

In [41]:
from keras.models import load_model

In [42]:
model = load_model("epochBIG.h5")

/Users/nishit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [43]:
tokenizer = load(open('epochBIG', 'rb'))

In [69]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=25)

"a wild fowl skin though i thought that ahab 's nine handfuls master is intense a weazel do that 's that i do n't get"